In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset

from utils.custom_plots import plot_stacked

In [4]:
from adtk.metrics import f1_score, precision, recall
from adtk.data import to_events

## Preparing dataset

In [5]:
ds = GhlKasperskyDataset()
ds.shake_not_stir()

## Train watchman

In [6]:
from utils.watchmen import LimitWatchman

In [7]:
watchman = LimitWatchman()

In [8]:
train_gen = ds.train_generator()
for data, fault_vector, info in train_gen:
    watchman.learn(data, tolerance=0.05)

In [9]:
watchman.limits

{'input_temp_gc': {'lo': 0.256703948974609, 'hi': 31.475138092041014},
 'rt_temp_gc': {'lo': 8.830712890624998, 'hi': 59.339147949218756},
 'ht_temp_gc': {'lo': 8.903743743896484, 'hi': 61.24625015258789},
 'ct_temp_gc': {'lo': -1.2788215637207045, 'hi': 58.73142776489258},
 'rt_level_m': {'lo': 0.44499999880790697, 'hi': 2.755000048875809},
 'ht_level_m': {'lo': 0.08749997038394214, 'hi': 0.6125000251457096},
 'ct_level_m': {'lo': -0.33062317012809217, 'hi': 13.975549965864047},
 'input_flow_state': {'lo': -0.025000000000000022, 'hi': 1.025},
 'supply_flow_state': {'lo': -0.025000000000000022, 'hi': 1.025},
 'return_flow_state': {'lo': -0.025000000000000022, 'hi': 1.025},
 'output_flow_state': {'lo': -0.025000000000000022, 'hi': 1.025},
 'heater_state': {'lo': -0.025000000000000022, 'hi': 1.025}}

## Examine watchman

In [20]:
test_gen = ds.test_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for data, fault_vector, info in tqdm(test_gen, total=48):
    detect_matrix = watchman.examine(data)
    detect_vector = detect_matrix.sum(axis=1)
    examine_list.loc[info, 'precision'] = precision(fault_vector, detect_vector)
    examine_list.loc[info, 'recall'] = recall(fault_vector, detect_vector)
    examine_list.loc[info, 'f1_score'] = f1_score(fault_vector, detect_vector)
    if max(fault_vector) and max(detect_vector):
        break

  0%|          | 0/48 [00:00<?, ?it/s]

In [21]:
plot_stacked(data,
             title=info,
             group='value_unit',
             faults=fault_vector,
             detect=detect_matrix,
            )